# Named Entity Extraction Tutorial
This tutorial is a slight modification of the tutorial by Sam Galen.

In [1]:
from __future__ import print_function
from sklearn.metrics import confusion_matrix
import io
import nltk
import scipy
import codecs
import sklearn
import pycrfsuite
import pandas as pd
from itertools import chain
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

print('sklearn version:', sklearn.__version__)
print('Libraries succesfully loaded!')


sklearn version: 0.21.2
Libraries succesfully loaded!


In [2]:
def sent2features(sent, feature_func):
    return [feature_func(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [s[-1] for s in sent]

def sent2tokens(sent):
    return [s[0] for s in sent]

def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(y_true)
    y_pred_combined = lb.transform(y_pred)
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )
            
def word2simple_features(sent, i):
    '''
    This makes a simple baseline.  
    You can add and/or remove features to get (much?) better results.
    Experiment with it as you will need to do this for assignment.
    '''
    word = sent[i][0]
    
    features = {
        'bias': 1.0, # This feature is constant for all words.
        'word.lower()': word.lower(), # This feature is the word, ignoring case.
        'word[-2:]': word[-2:], # This feature is the last two characters of the word (i.e. the suffix).
    }
    if i == 0:
        features['BOS'] = True # Mark the beginning of sentence.
        
    if i == len(sent)-1:
        features['EOS'] = True # Mark the end of sentence.

    return features

# load data and preprocess
def extract_data(path):
    """
    Extracting data from train file or test file. 
    path - the path of the file to extract
    
    return:
        res - a list of sentences, each sentence is a
              a list of tuples. For train file, each tuple
              contains token and label. For test file, each
              tuple only contains token.
        ids - a list of ids for the corresponding token. This
              is mainly for Kaggle submission.
    """
    file = io.open(path, mode="r", encoding="utf-8")
    next(file)
    res = []
    ids = []
    sent = []
    for line in file:
        if line != '\n':
            # Each line contains the position ID, the token, and (for the training set) the label.
            parts = line.strip().split(' ')
            sent.append(tuple(parts[1:]))
            ids.append(parts[0])
        else:
            res.append(sent)
            sent = []
                
    return res, ids
            

# Build a NER classifier

## Load data and extract features

In [3]:
# Load train and test data
train_data, train_ids = extract_data('train')
test_data, test_ids = extract_data('test')

# Load true labels for test data
test_labels = list(pd.read_csv('test_ground_truth').loc[:, 'label'])

print('Train and Test data loaded succesfully!')

# Feature extraction using the word2simple_features function
train_features = [sent2features(s, feature_func=word2simple_features) for s in train_data]
train_labels = [sent2labels(s) for s in train_data]
test_features = [sent2features(s, feature_func=word2simple_features) for s in test_data]

trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(train_features, train_labels):
    trainer.append(xseq, yseq)
print('Feature Extraction done!')    

# Explore the extracted features    
sent2features(train_data[0], word2simple_features)

Train and Test data loaded succesfully!
Feature Extraction done!


[{'bias': 1.0, 'word.lower()': 'también', 'word[-2:]': 'én', 'BOS': True},
 {'bias': 1.0, 'word.lower()': 'el', 'word[-2:]': 'el'},
 {'bias': 1.0, 'word.lower()': 'secretario', 'word[-2:]': 'io'},
 {'bias': 1.0, 'word.lower()': 'general', 'word[-2:]': 'al'},
 {'bias': 1.0, 'word.lower()': 'de', 'word[-2:]': 'de'},
 {'bias': 1.0, 'word.lower()': 'la', 'word[-2:]': 'la'},
 {'bias': 1.0, 'word.lower()': 'asociación', 'word[-2:]': 'ón'},
 {'bias': 1.0, 'word.lower()': 'española', 'word[-2:]': 'la'},
 {'bias': 1.0, 'word.lower()': 'de', 'word[-2:]': 'de'},
 {'bias': 1.0, 'word.lower()': 'operadores', 'word[-2:]': 'es'},
 {'bias': 1.0, 'word.lower()': 'de', 'word[-2:]': 'de'},
 {'bias': 1.0, 'word.lower()': 'productos', 'word[-2:]': 'os'},
 {'bias': 1.0, 'word.lower()': 'petrolíferos', 'word[-2:]': 'os'},
 {'bias': 1.0, 'word.lower()': ',', 'word[-2:]': ','},
 {'bias': 1.0, 'word.lower()': 'aurelio', 'word[-2:]': 'io'},
 {'bias': 1.0, 'word.lower()': 'ayala', 'word[-2:]': 'la'},
 {'bias': 1.

## Explore the classifier parameters

In [4]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

## Set the classifier parameters

In [5]:
trainer.set_params({
    'c1': 100.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

## Train a NER model

In [6]:
%%time
trainer.train('ner-esp.model')

print('Training done :)')

Training done :)
Wall time: 6.92 s


## Make predictions with your NER model
Make predictions and evaluate your model on the test set.
To use your NER model, create pycrfsuite.Tagger, open the model, and use the "tag" method, as follows:

In [7]:
# Make predictions
tagger = pycrfsuite.Tagger()
tagger.open('ner-esp.model')
test_pred = [tagger.tag(xseq) for xseq in test_features]
test_pred = [s for w in test_pred for s in w]

## Print evaluation
print(bio_classification_report(test_pred, test_labels))


              precision    recall  f1-score   support

       B-LOC       0.02      0.97      0.04        38
      B-MISC       0.00      0.17      0.00         6
      I-MISC       0.00      0.00      0.00      1872
       B-ORG       0.13      0.96      0.23       428
       I-ORG       0.00      0.13      0.00        39
       B-PER       0.02      0.78      0.03        40
       I-PER       0.03      0.88      0.06        58

   micro avg       0.04      0.22      0.07      2481
   macro avg       0.03      0.56      0.05      2481
weighted avg       0.03      0.22      0.04      2481
 samples avg       0.00      0.00      0.00      2481



C:\Program_Files\Anaconda3\envs\comp4650_2019\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Program_Files\Anaconda3\envs\comp4650_2019\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [8]:
print (len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'num': 50, 'scores': {}, 'loss': 100700.567394, 'feature_norm': 17.166387, 'error_norm': 1023.086536, 'active_features': 158, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.113}


## Check what the classifier has learned

In [9]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
B-PER  -> I-PER   4.910164
B-MISC -> I-MISC  4.361037
I-ORG  -> I-ORG   4.288836
I-MISC -> I-MISC  4.272204
B-ORG  -> I-ORG   3.881272
I-LOC  -> I-LOC   3.695069
I-PER  -> I-PER   3.513057
B-LOC  -> I-LOC   3.467629
O      -> B-ORG   1.969413
O      -> O       1.610842
O      -> B-LOC   1.551324
O      -> B-PER   1.270429
O      -> B-MISC  1.074687
B-LOC  -> O       0.494818
B-ORG  -> O       0.405177

Top unlikely transitions:
B-LOC  -> I-LOC   3.467629
O      -> B-ORG   1.969413
O      -> O       1.610842
O      -> B-LOC   1.551324
O      -> B-PER   1.270429
O      -> B-MISC  1.074687
B-LOC  -> O       0.494818
B-ORG  -> O       0.405177
I-PER  -> O       0.140915
I-ORG  -> O       -0.000078
I-MISC -> O       -0.167976
O      -> I-LOC   -1.803157
O      -> I-PER   -1.829670
O      -> I-MISC  -1.966156
O      -> I-ORG   -2.171883


We can see that, for example, it is very likely that the beginning of a person name (B-PER) will be followed by a token inside person name (I-PER). Also note O -> B-LOC are penalized.

## Check the state features

In [10]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
2.897303 O      EOS
2.750286 O      word.lower():el
2.681898 B-ORG  word.lower():efe
2.645789 B-ORG  word.lower():gobierno
2.338800 O      bias
2.304716 O      word.lower():una
2.083749 O      word.lower():en
2.068063 O      word.lower():la
2.064000 O      word.lower():,
2.064000 O      word[-2:]:,
1.898316 O      word.lower():que
1.896922 I-PER  word[-2:]:ez
1.710038 O      word.lower():al
1.636413 O      word.lower():y
1.550425 B-LOC  BOS
1.520263 B-ORG  word[-2:]:FE
1.496896 B-LOC  word[-2:]:id
1.443095 O      word.lower():para
1.437732 O      word.lower():por
1.428778 O      word.lower():a

Top negative:
-0.039968 O      word[-2:]:io
-0.040217 O      word[-2:]:co
-0.085603 O      word[-2:]:ca
-0.122929 B-PER  word.lower():la
-0.180838 I-ORG  bias
-0.207791 B-MISC bias
-0.217876 B-PER  word[-2:]:de
-0.222371 B-PER  word[-2:]:ón
-0.248464 O      word[-2:]:ga
-0.292959 O      word.lower():gobierno
-0.315779 B-ORG  word[-2:]:es
-0.367054 B-LOC  word[-2:]:de
-0.412633 O   